In [6]:
# ! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

In [7]:
#pip install python-dotenv

In [8]:
from dotenv import load_dotenv
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
load_dotenv()

True

In [9]:
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [10]:
import bs4 #html parsing
from langchain import hub # fetches prebuilt prompt from langchain hub
from langchain.text_splitter import RecursiveCharacterTextSplitter # splits long texts into chunks
from langchain_community.document_loaders import WebBaseLoader # loads web pages as LangChain documents
from langchain_community.vectorstores import Chroma # a local vector store db
from langchain_core.output_parsers import StrOutputParser #converts llm responses into plain strings
from langchain_core.runnables import RunnablePassthrough # passess the original output through a chain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings #chatllmwrapper #text to embedding vectors

#### INDEXING ####

# Load Documents: fetches the blog post and keeps only yhe element from class post-content,title and header
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split into chunks : break 1000 char chunks and 200 char overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed and store in vector db : openaiembedding turns chunk into vector, chroma.from_docs saves in local chroma dir(in memoery unless given a persist dir)
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever() # gives retriever interface that does similarity search with top k matches

#### RETRIEVAL and GENERATION ####

# Prompt 
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing : concatenates retrieved chunks into one context for the prompt
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("What is Task Decomposition?")


USER_AGENT environment variable not set, consider setting it to identify your requests.


'Task Decomposition is a technique used to break down complex tasks into smaller and simpler steps, allowing for easier execution and understanding. It can be achieved through methods such as prompting with specific instructions, human inputs, or utilizing external classical planners for long-horizon planning. Task decomposition helps in transforming big tasks into manageable ones, enhancing model performance and interpretation of the thinking process.'